In [12]:
import mysql.connector
import openai
from dotenv import load_dotenv
import os
import logging
import time
from functools import wraps
from pathlib import Path 

In [13]:
# Configurar logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Specify the path to .env file in the project root
env_path = Path('../.env')
load_dotenv(dotenv_path=env_path)


# Check if the environment variables are loaded
db_name = os.getenv('DB_NAME')
print(f"Database Name: {db_name}")

if db_name is None:
    print("Error: Could not load .env file or DB_NAME is not defined")
else:
    print(f"Loaded database name: {db_name}")

# Decorador para medir el tiempo de ejecución de las funciones
def time_execution(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        execution_time = end_time - start_time
        logging.info(f"Function '{func.__name__}' executed in {execution_time:.4f} seconds")
        return result
    return wrapper

# Decorador para capturar excepciones y registrarlas
def log_exceptions(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        try:
            return func(*args, **kwargs)
        except Exception as e:
            logging.error(f"Error occurred in function '{func.__name__}': {e}")
            raise e
    return wrapper

Database Name: CustomerSales
Loaded database name: CustomerSales


In [14]:
# Clase para gestionar la conexión a MySQL
class MySQLDatabase:
    def __init__(self):
        self.host = os.getenv('DB_HOST')
        self.user = os.getenv('DB_USER')
        self.password = os.getenv('DB_PASSWORD')
        self.database = os.getenv('DB_NAME')
        self.port = os.getenv('DB_PORT')
        self.connection = None

    @log_exceptions
    @time_execution
    def connect(self):
        logging.info("Connecting to MySQL database...")
        self.connection = mysql.connector.connect(
            host=self.host,
            user=self.user,
            password=self.password,
            database=self.database,
            port=self.port
        )
        logging.info("Successfully connected to the database.")

    @log_exceptions
    @time_execution
    def execute_query(self, query):
        if self.connection is None:
            self.connect()
        logging.info(f"Executing query: {query}")
        cursor = self.connection.cursor()
        cursor.execute(query)
        result = cursor.fetchall()
        logging.info("Query executed successfully.")
        return result

    @log_exceptions
    @time_execution
    def close(self):
        if self.connection:
            self.connection.close()
            logging.info("Connection closed.")

In [15]:
# Crear instancia de la base de datos
db = MySQLDatabase()

In [16]:
# Conectar a la base de datos
db.connect()

2024-09-22 00:57:00,050 - INFO - Connecting to MySQL database...
2024-09-22 00:57:00,066 - INFO - Successfully connected to the database.
2024-09-22 00:57:00,067 - INFO - Function 'connect' executed in 0.0169 seconds


In [17]:
# ejecutar una consulta SQL
query = "SELECT * FROM ClientesVentas LIMIT 5;"
result = db.execute_query(query)
print(result)

2024-09-22 00:57:00,610 - INFO - Executing query: SELECT * FROM ClientesVentas LIMIT 5;
2024-09-22 00:57:00,616 - INFO - Query executed successfully.
2024-09-22 00:57:00,616 - INFO - Function 'execute_query' executed in 0.0059 seconds


[(1, 'Juan', 'Pérez', 35, 'Apple', Decimal('1200.50')), (2, 'Ana', 'García', 28, 'Samsung', Decimal('950.75')), (3, 'Luis', 'Martínez', 42, 'Sony', Decimal('560.00')), (4, 'Carmen', 'Fernández', 30, 'LG', Decimal('1340.10')), (5, 'Pedro', 'López', 45, 'Apple', Decimal('2500.00'))]


In [18]:
from openai import OpenAI
import os

class NaturalLanguageToSQL:
    def __init__(self):
        self.client = openai.OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

    def translate(self, natural_language_query):
        messages = [
            {"role": "system", "content": "Eres un asistente que traduce consultas en lenguaje natural a SQL usando la tabla ClientesVentas, que contiene las columnas: id, nombre, apellidos, edad, marca_producto e importe_vendido."},
            {"role": "user", "content": natural_language_query}
        ]
        response = self.client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            max_tokens=100,
            temperature=0
        )
        sql_query = response.choices[0].message.content.strip()
    
        # Limpiar la consulta eliminando caracteres innecesarios
        if sql_query.startswith("```sql"):
            sql_query = sql_query[5:]
        if sql_query.endswith("```"):
            sql_query = sql_query[:-3]
    
        # Eliminar caracteres adicionales que puedan estar fuera de lugar
        sql_query = sql_query.lstrip("l").strip()  # Eliminar el 'l' al inicio de la consulta si aparece
        
        return sql_query.strip()

In [19]:
# Clase principal para gestionar el flujo completo
class QueryProcessor:
    def __init__(self):
        self.db = MySQLDatabase()
        self.nl_to_sql = NaturalLanguageToSQL()

    @log_exceptions
    @time_execution
    def process_query(self, natural_language_query):
        # 1. Convertir el lenguaje natural a SQL
        sql_query = self.nl_to_sql.translate(natural_language_query)
        logging.info(f"SQL Query: {sql_query}")
        
        # 2. Ejecutar la consulta SQL
        result = self.db.execute_query(sql_query)
        
        # 3. Devolver el resultado y la consulta SQL generada
        return {"sql_query": sql_query, "result": result}

    @log_exceptions
    @time_execution
    def close(self):
        # Cerrar la conexión a la base de datos
        self.db.close()

In [20]:
# Crear instancia del procesador de consultas
query_processor = QueryProcessor()

In [23]:
# Your natural language query
natural_language_query = "Show me average importe_vendido of customers"

# Procesar la consulta
result = query_processor.process_query(natural_language_query)
print(result)

2024-09-22 00:59:00,696 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-22 00:59:00,702 - INFO - SQL Query: SELECT AVG(importe_vendido) AS promedio_importe_vendido
FROM ClientesVentas;
2024-09-22 00:59:00,749 - INFO - Executing query: SELECT AVG(importe_vendido) AS promedio_importe_vendido
FROM ClientesVentas;
2024-09-22 00:59:00,772 - INFO - Query executed successfully.
2024-09-22 00:59:00,773 - INFO - Function 'execute_query' executed in 0.0246 seconds
2024-09-22 00:59:00,774 - INFO - Function 'process_query' executed in 1.2476 seconds


{'sql_query': 'SELECT AVG(importe_vendido) AS promedio_importe_vendido\nFROM ClientesVentas;', 'result': [(Decimal('1126.453333'),)]}


In [24]:
# Imprimir el resultado
sql_query = result["sql_query"]
query_result = result["result"]
print(f"SQL Query: {sql_query}")
print(f"Query Result: {query_result}")

SQL Query: SELECT AVG(importe_vendido) AS promedio_importe_vendido
FROM ClientesVentas;
Query Result: [(Decimal('1126.453333'),)]


In [10]:
# Your natural language query
natural_language_query = "Show me the names and surnames of customers older than 30 years and who sold more than 1000 euros."

# Procesar la consulta
result = query_processor.process_query(natural_language_query)
print(result)

2024-09-22 00:54:49,842 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-22 00:54:49,845 - INFO - SQL Query: SELECT nombre, apellidos 
FROM ClientesVentas 
WHERE edad > 30 AND importe_vendido > 1000;
2024-09-22 00:54:49,846 - INFO - Connecting to MySQL database...
2024-09-22 00:54:49,861 - INFO - Successfully connected to the database.
2024-09-22 00:54:49,862 - INFO - Function 'connect' executed in 0.0160 seconds
2024-09-22 00:54:49,862 - INFO - Executing query: SELECT nombre, apellidos 
FROM ClientesVentas 
WHERE edad > 30 AND importe_vendido > 1000;
2024-09-22 00:54:49,864 - INFO - Query executed successfully.
2024-09-22 00:54:49,864 - INFO - Function 'execute_query' executed in 0.0181 seconds
2024-09-22 00:54:49,864 - INFO - Function 'process_query' executed in 0.7860 seconds


{'sql_query': 'SELECT nombre, apellidos \nFROM ClientesVentas \nWHERE edad > 30 AND importe_vendido > 1000;', 'result': [('Juan', 'Pérez'), ('Pedro', 'López'), ('Marta', 'Sánchez'), ('Isabel', 'Mendoza'), ('Carlos', 'Jiménez')]}


In [11]:
# Imprimir el resultado
sql_query = result["sql_query"]
query_result = result["result"]
print(f"SQL Query: {sql_query}")
print(f"Query Result: {query_result}")

SQL Query: SELECT nombre, apellidos 
FROM ClientesVentas 
WHERE edad > 30 AND importe_vendido > 1000;
Query Result: [('Juan', 'Pérez'), ('Pedro', 'López'), ('Marta', 'Sánchez'), ('Isabel', 'Mendoza'), ('Carlos', 'Jiménez')]


In [20]:
# Cerrar la conexión
query_processor.close()

2024-09-22 00:46:04,354 - INFO - Connection closed.
2024-09-22 00:46:04,355 - INFO - Function 'close' executed in 0.0014 seconds
2024-09-22 00:46:04,355 - INFO - Function 'close' executed in 0.0017 seconds
